# Running an Private Ethereum Cluster on HPC

In [1]:
!wget https://gethstore.blob.core.windows.net/builds/geth-linux-amd64-1.8.14-316fc7ec.tar.gz

--2018-08-25 11:05:12--  https://gethstore.blob.core.windows.net/builds/geth-linux-amd64-1.8.14-316fc7ec.tar.gz
Resolving gethstore.blob.core.windows.net (gethstore.blob.core.windows.net)... 40.113.27.176
Connecting to gethstore.blob.core.windows.net (gethstore.blob.core.windows.net)|40.113.27.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11617405 (11M) [application/octet-stream]
Saving to: ‘geth-linux-amd64-1.8.14-316fc7ec.tar.gz’

100%[======================================>] 11,617,405  4.97MB/s   in 2.2s   

2018-08-25 11:05:16 (4.97 MB/s) - ‘geth-linux-amd64-1.8.14-316fc7ec.tar.gz’ saved [11617405/11617405]



# Init configuration for Ethereum node

Sources: 

* <https://hackernoon.com/setup-your-own-private-proof-of-authority-ethereum-network-with-geth-9a0a3750cda8>
* https://medium.com/@niceoneallround/creating-a-private-ethereum-network-on-a-mac-c417ab971055
* https://github.com/ethereum/go-ethereum/wiki/Managing-your-accounts


In [1]:
!geth version

Geth
Version: 1.8.14-stable
Git Commit: 316fc7ecfc10d06603f1358c1f4c1020ec36dd2a
Architecture: amd64
Protocol Versions: [63 62]
Network Id: 1
Go Version: go1.10.3
Operating System: linux
GOPATH=
GOROOT=/home/travis/.gimme/versions/go1.10.3.linux.amd64


In [3]:
!geth --datadir node1/ --password init_password.txt account new

INFO [08-26|03:37:19.880] Maximum peer count                       ETH=25 LES=0 total=25
Address: {dcc69d06fe344e68338e0489ba9892c5376e5f8f}


In [4]:
!geth --datadir node2/ --password init_password.txt account new

INFO [08-26|03:37:38.213] Maximum peer count                       ETH=25 LES=0 total=25
Address: {1d87ddafac8337659d173a6b8d2f77652f89b78f}


In [6]:
!cp init_password.txt node1/password.txt
!cp init_password.txt node2/password.txt

## Run Puppeth to generate configuration

Generate file PilotNet.json and initialize Genisis Block for both nodes

In [1]:
!geth --datadir node1/ init PilotNet.json
!geth --datadir node2/ init PilotNet.json

INFO [08-26|04:29:36.897] Maximum peer count                       ETH=25 LES=0 total=25
INFO [08-26|04:29:36.897] Allocated cache and file handles         database=/home/01131/tg804093/notebooks/blockchain/node1/geth/chaindata cache=16 handles=16
INFO [08-26|04:29:36.923] Persisted trie from memory database      nodes=355 size=51.91kB time=1.080013ms gcnodes=0 gcsize=0.00B gctime=0s livenodes=1 livesize=0.00B
INFO [08-26|04:29:36.924] Successfully wrote genesis state         database=chaindata                                                      hash=e2f6bc…952ee5
INFO [08-26|04:29:36.924] Allocated cache and file handles         database=/home/01131/tg804093/notebooks/blockchain/node1/geth/lightchaindata cache=16 handles=16
INFO [08-26|04:29:36.941] Persisted trie from memory database      nodes=355 size=51.91kB time=1.091658ms gcnodes=0 gcsize=0.00B gctime=0s livenodes=1 livesize=0.00B
INFO [08-26|04:29:36.942] Successfully wrote genesis state         database=lightchaindata        

Create static Nodes file and put into data dir of each node:

    cat node1/static-nodes.json
    [         "enode://b5e9814fb254d792947ab238d31694be0b79bb627fb22fdf545a29de142cff90d0ff88ea67e7fe5e052c2a7fa6170cfc5c23e5f52a36bfa7e69bcd3075a3fb8e@129.114.58.133:30304?discport=0",
            "enode://d8505a82f17483aeff5a57517b6295e3879d67d52e72e6f408c741f314dda0e2530e7c3809ef284d5dd2086faf4f545e0b1dbaf0969cd0a82ad143ec5a6f9c82@129.114.58.101:30305?discport=0"
            ]

       cat node2/static-nodes.json
            [
              "enode://b5e9814fb254d792947ab238d31694be0b79bb627fb22fdf545a29de142cff90d0ff88ea67e7fe5e052c2a7fa6170cfc5c23e5f52a36bfa7e69bcd3075a3fb8e@129.114.58.133:30304?discport=0",
            "enode://d8505a82f17483aeff5a57517b6295e3879d67d52e72e6f408c741f314dda0e2530e7c3809ef284d5dd2086faf4f545e0b1dbaf0969cd0a82ad143ec5a6f9c82@129.114.58.101:30305?discport=0"
            ]

In [ ]:
# ENODE1 port 30304, rpcport 8504, ipc node1/geth.ipcpatch
!geth --datadir node1 \
      --networkid 31269 \
      --port 30304 \
      --nodiscover \
      --rpc \
      --rpcaddr 0.0.0.0 \
      --rpcport 8504 \
      --ipcpath node1/geth.ipc \
      --unlock 0xdcc69d06fe344e68338e0489ba9892c5376e5f8f \
      --password node1/password.txt \
      --mine --minerthreads 1 --gasprice "10000" \
      --etherbase 0xdcc69d06fe344e68338e0489ba9892c5376e5f8f \
      --verbosity 3

In [ ]:
# ENODE2 on port 30305, rpcport 8505, ipc node2/geth.ipcpatch
!geth --datadir node2 \
      --networkid 31269 \
      --port 30305 \
      --nodiscover \
      --rpc \
      --rpcaddr 0.0.0.0 \
      --rpcport 8505 \
      --ipcpath node2/geth.ipc \
      --unlock 0x1d87ddafac8337659d173a6b8d2f77652f89b78f
      --password node2/password.txt \
      --mine --minerthreads 1 --gasprice "10000" \
      --etherbase 0x1d87ddafac8337659d173a6b8d2f77652f89b78f \
      --verbosity 3